In [ ]:
from src.utils.trust_region import TrustRegion
from src.utils.lagrange_polynomial import LagrangePolynomials
import matplotlib.pyplot as plt
import numpy as np
import casadi as ca

def myfunc(x:np.ndarray) -> np.ndarray: # Rosenbrock function: OF
    return 1*(x[1]-x[0]**2)**2+((x[0]-1)**2)/100

def eq_constraint(x:np.ndarray) -> np.ndarray: # equality constraints
    return x[1] - x[0]

dataset = np.array([[0.0, 0.0], [1.0, 0.0], [0.0, 1.0], [-1.0, 0.0], [1.0, 1.0]]).T + np.array([[2.2], [2.2]])
INPUT_SYMBOL = ca.SX.sym('x', dataset.shape[0])

poly_mf = LagrangePolynomials(input_symbols=INPUT_SYMBOL, pdegree=2)
poly_mceq = LagrangePolynomials(input_symbols=INPUT_SYMBOL, pdegree=2)
mf = []
mceq = []
for i in range(dataset.shape[1]):
    x = dataset[:, i]
    mf.append(myfunc(x))
    mceq.append(eq_constraint(x))
mf = np.array(mf)
mceq = np.array(mceq)
poly_mf.initialize(v=dataset, f=mf)
poly_mceq.initialize(v=dataset, f=mceq)

tr = TrustRegion(input_symbols=INPUT_SYMBOL, dataset=dataset, results=mf, func=myfunc, eq_const=[poly_mceq])
tr.run(max_radius=1.5, max_iter=20, rad_tol=1E-5)

In [ ]:
from casadi import *
import glob

x, y = np.meshgrid(np.linspace(-4, 6, 100),
                    np.linspace(-4, 6, 100))
    
levels = list(np.arange(0, 5, 1))

old_center = [0,0]
old_radius = 0
    
files = glob.glob('./plots/*')
for f in files:
    os.remove(f)
        
for i, m in enumerate(tr.list_of_models):
    if i % 1 == 0:
        center = m.sample_set.ball.center
        radius = tr.list_of_radius[i]
        intx = x - center[0]
        inty = y - center[1]
        dist = np.sqrt(intx**2 + inty**2)
        intindices = dist <= radius

        fig, ax = plt.subplots(1)
        ax.set_title(f"{tr.list_of_status[i]['criticality_step']} \n {tr.list_of_status[i]['acceptance']}")
        func = myfunc([x, y])
        # func[intindices] = m.model_polynomial.feval(x, y)[intindices]
        func_map = m.model_polynomial.feval.map(x.shape[0]*x.shape[1])
        x_dm = DM(x).reshape((1, x.shape[0]*x.shape[1]))
        y_dm = DM(y).reshape((1, x.shape[0]*x.shape[1]))
        # print(func_map, vertcat(x_dm, y_dm).shape)
        func_dm = func_map(vertcat(x_dm, y_dm))
        func_dm = func_dm.reshape((x.shape[0], x.shape[1])).full()
        func[intindices] = func_dm[intindices]
        
        
        circle1 = plt.Circle(center, radius, color='black', fill=False)
        circle2 = plt.Circle(old_center, old_radius, color='red', fill=False)
        
        ax.add_patch(circle1)
        ax.add_patch(circle2)
        ax.contour(x, y, func, levels)
        plt.scatter(m.y[0, :], m.y[1, :], label=f'iteration_{i}')
        plt.scatter(m.y[0, 0], m.y[1, 0], label=f'Best point')
        plt.plot([-6,6], [-6,6], 'b-', label=f"constraint")
        plt.legend()
        plt.savefig(f"./plots/TR_plots_{i}.png")
        plt.close()

        old_center = center*1
        old_radius = radius*1

        
## Plot of objective function
fig, ax = plt.subplots(1)
ax.set_title(f"Objective Function vs Iterations")
ax.scatter(range(len(tr.list_of_OF)), tr.list_of_OF, color="black", label="OF")
try:
    ax.set_yscale("log")
except:
    ax.set_yscale("linear")

plt.legend()
plt.savefig(f"./plots/score_OF.png")
plt.close()

eq_val = []
## Plot of constraint violation
for i in range(len(tr.list_of_OF)):
    x = tr.list_of_models[i].y
    eq_val.append(np.min(np.abs(eq_constraint(x))))
    
fig, ax = plt.subplots(1)
ax.set_title(f"Min constraint vs Iterations")
ax.scatter(range(len(tr.list_of_OF)), eq_val, color="black", label="OF")
try:
    ax.set_yscale("log")
except:
    ax.set_yscale("linear")

plt.legend()
plt.savefig(f"./plots/score_violation.png")
plt.close()

In [ ]:
for i in range(len(tr.list_of_OF)):
    x = tr.list_of_models[i].y
    const = eq_constraint(x)
    print(const)